In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define number of classes and class labels
num_classes = 5
class_names = ['Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset path
data_path = r"C:\Users\HP\Downloads\gaussian_filtered_images\sample"

# Load dataset
dataset = ImageFolder(root=data_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load pretrained ResNet50 model
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)

# Freeze earlier layers
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier head
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")


Using device: cpu
Epoch [1/10], Loss: 6.3263, Accuracy: 21.60%
Epoch [2/10], Loss: 5.5141, Accuracy: 54.40%
Epoch [3/10], Loss: 4.6405, Accuracy: 66.40%
Epoch [4/10], Loss: 3.9311, Accuracy: 70.40%
Epoch [5/10], Loss: 3.0876, Accuracy: 83.20%
Epoch [6/10], Loss: 2.7802, Accuracy: 83.20%
Epoch [7/10], Loss: 2.2857, Accuracy: 87.20%
Epoch [8/10], Loss: 2.1771, Accuracy: 88.80%
Epoch [9/10], Loss: 1.6899, Accuracy: 94.40%
Epoch [10/10], Loss: 1.4641, Accuracy: 92.80%


In [5]:
# Save the model after training
model_save_path = r"C:\Users\HP\Downloads\gaussian_filtered_images\retina.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved successfully at {model_save_path}")

Model saved successfully at C:\Users\HP\Downloads\gaussian_filtered_images\retina.pth


In [6]:
# Function to load the model for prediction
def load_model():
    # Load ResNet50 with the modified last layer
    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    
    # Load the saved model weights
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

# Load the model
model = load_model()

In [7]:
def predict_image(img_path):
    model.eval()
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        class_name = class_names[predicted.item()]

    print(f"Predicted Class: {class_name}")

In [12]:
folder_path = r"C:\Users\HP\Downloads\gaussian_filtered_images\predict"
image_name = "C:/Users/HP/OneDrive/Desktop/demo 2R.jpg"  # Example image name
image_path = os.path.join(folder_path, image_name)

predict_image(image_path)

Predicted Class: Moderate
